<a href="https://colab.research.google.com/github/Pugianf/ANTAQ/blob/main/An%C3%A1lise_IDA_Indicadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Data Analysis of IDA Indicators</h1>

### Settings


In [1]:
# importing libraries

import pandas as pd
import numpy as np

In [2]:
# connecting google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing CSV

In [15]:
# getting data from IDA

df_v1 = pd.read_csv('/content/drive/MyDrive/ANTAQ/Datasets/IDA/database_IDAv1.csv', encoding='latin1', sep=';')

df_v2 = pd.read_csv('/content/drive/MyDrive/ANTAQ/Datasets/IDA/database_IDAv2.csv', encoding='latin1', sep=';')

In [16]:
# getting code of ports

df_portos = pd.read_csv('/content/drive/MyDrive/ANTAQ/Datasets/IDA/Portos.csv', encoding='latin1', sep=';')

df_tup = pd.read_csv('/content/drive/MyDrive/ANTAQ/Datasets/IDA/IDTerminal.csv', encoding='latin1', sep=';')

### Wrangling Database

In [17]:
# setting the same column names in df_portos and df_tup to create a unique df_instalacoes

df_portos = df_portos[['cdtrigrama', 'nome']].copy()
df_portos.rename(columns={'cdtrigrama': 'Cod'}, inplace=True)
df_tup.rename(columns={'IDTerminal': 'Cod', 'NOTerminal': 'nome'}, inplace=True)
df_instalacoes = pd.concat([df_portos, df_tup])

In [18]:
# checking NaN values and treating them

print('Is there any NaN values in df_v1?', df_v1['DSAtributoQuestionario'].isnull().values.any())
print('How many NaN values in df_v1?', df_v1['DSAtributoQuestionario'].isnull().sum())

print('Is there any NaN values in df_v2?', df_v2.isnull().values.any())

# Creating a column to explicit NaN values with 'Yes' or 'No'

df_v1.loc[df_v1['DSAtributoQuestionario'].isnull(),'value_is_NaN'] = 'Yes'
df_v1.loc[df_v1['DSAtributoQuestionario'].notnull(), 'value_is_NaN'] = 'No'

# dropping NaN values

df_v1.dropna(subset = ['DSAtributoQuestionario'], inplace=True)

Is there any NaN values in df_v1? True
How many NaN values in df_v1? 1609
Is there any NaN values in df_v2? False


In [19]:
# checking type of columns in df_v2

print(df_v2.dtypes)

# filtering df_v2 for 'IDTipoAtributo' == 0

df_v2 = df_v2[df_v2.IDTipoAtributo == 0].copy()

# dropping 'IDTipoAtributo' and 'NRPeso'

df_v2.drop(columns=['IDTipoAtributo', 'NRPeso', 'NRCategoriaQuestionario', 'NRIndicadorGlobalQuestionario'], inplace=True)

CDInstalacaoPortuaria                       object
IDTipoAtributo                               int64
NOAtributoQuestionario                      object
NRPeso                                      object
NRIndicadorEspecificoQuestionario           object
NOTituloIndicadorEspecificoQuestionario     object
NRCategoriaQuestionario                    float64
NRIndicadorGlobalQuestionario               object
NOIndicadorGlobalQuestionario               object
NOCategoriaQuestionario                     object
NOAvaliacao                                 object
dtype: object


In [20]:
# creating a dictionary to map values in indicator column

dict_indicador = {'1.1.1.1 Qual é a situação atual do Licenciamento Ambiental do porto? (C111)': 'Licenciamento Ambiental',
                  '1.1.1.2 Quantos profissionais de interesse no Núcleo Ambiental? (C112)': 'Quantidade e Qualidade dos Técnicos',
                  '1.1.1.3 Quantos treinamentos/capacitações ambientais os funcionários do Núcleo Ambiental recebem por ano? (C113)': 'Treinamento e Capacitação Ambiental',
                  '1.1.1.4 Qual a situação atual da Auditoria Ambiental do porto? (C114)': 'Auditoria Ambiental',
                  '1.1.2.1 O porto possui banco de dados oceanográficos/hidrológicos e meteorológicos/climatológicos próprio? (C121)': 'Base de Dados Oceano Meteorológicas',
                  '1.1.2.2 Quantos planos foram elaborados e implementados no porto em relação à prevenção de riscos e atendimento a emergência? (C122)': 'Prevenção de Riscos',
                  '1.1.2.3 Quantos acidentes ambientais ocorreram na Área do Porto Organizado (APO) e em seus arredores no último ano? (C123)': 'Ocorrência de Acidentes Ambientais',
                  '1.1.3.1 Quais as ações de retirada dos resíduos de navios no porto? (C131)': 'Ações de Retirada de Resíduos do Navio',
                  '1.1.3.2 Quais as ações relacionadas às operações de movimentação e armazenamento de contêineres com produtos perigosos? (C132)': 'Cargas Perigosas',
                  '1.1.4.1 O porto busca reduzir o seu consumo de energia? (C141)': 'Consumo e Eficiência no Uso de Energia',
                  '1.1.4.2 Há geração de energia limpa e renovável pelo porto para a sua utilização em atividades operacionais e/ou administrativas? (C142)': 'Tipos de Energia Utilizados',
                  '1.1.4.3 O porto dispõe de sistema para fornecimento de energia (Onshore Power Supply - OPS) aos navios? (C143)': 'Fornecimento de Energia para Embarcações',
                  '1.1.5.1 Como a internalização dos custos ambientais no orçamento é feita pelo porto? (C151)': 'Internalização dos Custos Ambientais',
                  '1.1.6.1 Qual o nível de divulgação de informações ambientais do porto através do seu sítio eletrônico na Internet? (C161)': 'Comunicação das Ações Ambientais',
                  '1.1.6.2 Em qual situação se encontra a Agenda Ambiental Local do porto? (C162)': 'Agenda Ambiental Local',
                  '1.1.6.3 Qual a situação atual da Agenda Ambiental Institucional do porto? (C163)': 'Agenda Ambiental Institucional',
                  '1.1.6.4 Quantas certificações voluntárias o porto possui? (C164)': 'Certificações Voluntárias',
                  '1.1.7.1 A Autoridade Portuária acompanha e controla o desempenho ambiental dos terminais arrendados e/ou operadores avulsos? (C171)': 'Papel da Autoridade Portuária',
                  '1.1.7.2 Qual a situação do licenciamento ambiental das empresas (terminais arrendados e/ou operadores avulsos)? (C172)': 'Licenciamento Ambiental das Empresas ',
                  '1.1.7.3 Qual a situação do Plano de Emergência Individual - PEI dos terminais arrendados? (C173)': 'PEI dos Terminais',
                  '1.1.7.4 Qual a situação das auditorias ambientais dos terminais arrendados? (C174)': 'Auditoria Ambiental dos Terminais',
                  '1.1.7.5 Qual a situação dos Planos de Gerenciamento de Resíduos Sólidos - PGRS dos terminais arrendados? (C175)': 'PGRS dos Terminais',
                  '1.1.7.6 Existem programas de educação ambiental promovidos pelos terminais arrendados? (C176)': 'Programa de EA nos Terminais',
                  '1.1.7.7 Qual a situação das certificações voluntárias dos terminais arrendados? (C177)': 'Certificação das Empresas',
                  '1.2.1.1 Há no porto a promoção de ações de educação ambiental? (C211)': 'Promoção de Ações de EA',
                  '1.2.2.1 Quantas ações de promoção da saúde existem no porto? (C221)': 'Ações de Promoção da Saúde',
                  '1.3.1.1 O porto realiza o monitoramento da qualidade da água do ambiente onde está inserido?  (C311)': 'Qualidade Ambiental do Corpo Hídrico',
                  '1.3.1.2 Há drenagem pluvial no porto? (C312)': 'Drenagem Pluvial',
                  '1.3.1.3 Qual a situação das ações de redução e reuso da água no porto? (C313)': 'Ações para Redução e Reúso da Água',
                  '1.3.2.1 Qual a situação atual da área dragada e sua disposição no porto? (C321)': 'Área Dragada e de Disposição',
                  '1.3.2.2 Há passivos ambientais relacionados no porto? (C322)': 'Passivos Ambientais',
                  '1.3.3.1 Há monitoramento dos poluentes atmosféricos (gases e particulado) emitidos pelo porto? (C331)': 'Poluentes Atmosféricos',
                  '1.3.3.2 Há monitoramento da poluição sonora emitida pelo porto? (C332)': 'Poluição Sonora',
                  '1.3.4.1 Há gerenciamento de resíduos sólidos no porto? (C341)': 'PGRS',
                  '1.4.1.1 Há monitoramento da fauna e flora no porto? (C411)': 'Monitoramento de Fauna e Flora',
                  '1.4.1.2 Existe no porto um Programa Integrado de Controle e Monitoramento da Fauna Sinantrópica Nociva? (C412)': 'Animais Sinantrópicos',
                  '1.4.1.3 Há algum levantamento ou monitoramento das espécies aquáticas exóticas/invasoras no porto? (C413)': 'Espécies Exóticas/Invasoras',
                  '1.2.2.2 Há plano de contigência de saúde no porto? (C222)': 'Planos de Contingência de Saúde',
                  '1.1.7.7 Existem programas de educação ambiental promovidos pelos terminais arrendados? (C177)': np.nan,
                  '1.1.7.6 Qual a situação das certificações voluntárias dos terminais arrendados? (C176)': np.nan}

In [21]:
df_v1.replace({'DSIndicadorEspecificoQuestionario': dict_indicador})

,CDTrigrama,DSAtributoQuestionario,NOAtributoQuestionario,DSIndicadorEspecificoQuestionario,DSIndicadorGeralQuestionario,DSCategoriaQuestionario,NOQUESTIONARIO,value_is_NaN
0,MCP,O porto possui Licença de Operação Válida e vi...,N5,Licenciamento Ambiental,1.1.1 GOVERNANÇA AMBIENTAL (C11),CATEGORIA ECONÔMICO-OPERACIONAIS (C1),2012.2,No
1,MCP,Possui o número mínimo de profisionais no Núcl...,N2,Quantidade e Qualidade dos Técnicos,1.1.1 GOVERNANÇA AMBIENTAL (C11),CATEGORIA ECONÔMICO-OPERACIONAIS (C1),2012.2,No
2,MCP,Não participaram de treinamento/capacitação ou...,N1,Treinamento e Capacitação Ambiental,1.1.1 GOVERNANÇA AMBIENTAL (C11),CATEGORIA ECONÔMICO-OPERACIONAIS (C1),2012.2,No
3,MCP,Nunca foi realizada auditoria ambiental.,N1,Auditoria Ambiental,1.1.1 GOVERNANÇA AMBIENTAL (C11),CATEGORIA ECONÔMICO-OPERACIONAIS (C1),2012.2,No
4,MCP,Existe banco de dados com atualização mínima s...,N3,Base de Dados Oceano Meteorológicas,1.1.2 SEGURANÇA (C12),CATEGORIA ECONÔMICO-OPERACIONAIS (C1),2012.2,No
...,...,...,...,...,...,...,...,...
8878,VIX,Atende duas das opções do N5.,N3,PGRS,1.3.4 GERENCIAMENTO DE RESÍDUOS SÓLIDOS (C34),CATEGORIA FÍSICO-QUÍMICOS (C3),2015.1,No
8879,VIX,( ) Foi realizada a caracterização da biota ...,N2,Monitoramento de Fauna e Flora,1.4.1 BIODIVERSIDADE (C41),CATEGORIA BIOLÓGICO-ECOLÓGICOS (C4),2015.1,No
8880,VIX,Atende três das opções do N5.,N4,Animais Sinantrópicos,1.4.1 BIODIVERSIDADE (C41),CATEGORIA BIOLÓGICO-ECOLÓGICOS (C4),2015.1,No
8881,VIX,Atende uma das opções do N3.,N1,Espécies Exóticas/Invasoras,1.4.1 BIODIVERSIDADE (C41),CATEGORIA BIOLÓGICO-ECOLÓGICOS (C4),2015.1,No


In [ ]:
# dropping the special year for Cabedelo

df = df[df.Ano != '2021 Cabedelo']

In [ ]:
# checking the years enrolled after dropping

years = sorted(list(set(df.Ano)))

years

['2015.2', '2016.1', '2017', '2018', '2019', '2020', '2021']